## Vanishing/Exploding Gradients

In [1]:
from tensorflow import keras

In [2]:
keras.layers.Dense(10, activation="relu", 
                    kernel_initializer="he_normal")

In [4]:
he_avg_init = keras.initializers.VarianceScaling(scale=2, 
                                                mode="fan_avg", 
                                                distribution="uniform")
keras.layers.Dense(10, activation="relu", 
                    kernel_initializer=he_avg_init)

In [5]:
# using leaky relu
leaky_relu = keras.layers.LeakyReLU(alpha=0.2)
keras.layers.Dense(10, activation=leaky_relu, 
                    kernel_initializer="he_normal")

In [6]:
keras.layers.Dense(10, activation="selu", 
                    kernel_initializer="lecun_normal")

In [8]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(300, activation="elu", 
                        kernel_initializer="he_normal"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(100, activation="elu", 
                        kernel_initializer="he_normal"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(10, activation="softmax")
])

In [9]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization_3 (Batc  (None, 784)              3136      
 hNormalization)                                                 
                                                                 
 dense_7 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_4 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_8 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_5 (Batc  (None, 100)             

In [10]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization_3/gamma:0', True),
 ('batch_normalization_3/beta:0', True),
 ('batch_normalization_3/moving_mean:0', False),
 ('batch_normalization_3/moving_variance:0', False)]

In [11]:
model.layers[1].updates

C:\Users\SG704\AppData\Local\Temp/ipykernel_12816/3873162892.py:1: UserWarning: `layer.updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  model.layers[1].updates


[]

In [2]:
from tensorflow import keras

In [3]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(300, 
        kernel_initializer="he_normal", 
        use_bias=False), 
    keras.layers.BatchNormalization(), 
    keras.layers.Activation("elu"), 
    keras.layers.Dense(100, 
        kernel_initializer="he_normal", 
        use_bias=False), 
    keras.layers.BatchNormalization(), 
    keras.layers.Activation("elu"), 
    keras.layers.Dense(10, activation="softmax")
])

In [4]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)

## Reusing Pretrained Layers

In [1]:
from tensorflow import keras
import numpy as np

def get_data_without_5_6(data, labels):
    new_data, new_labels = [], []
    for i, label in enumerate(labels):
        if (label != 5) & (label != 6):
            if label > 5:
                new_labels.append(labels[i]-2)
            else:
                new_labels.append(labels[i])
            new_data.append(data[i])
    return np.array(new_data), np.array(new_labels)
        
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_A, y_train_A = get_data_without_5_6(X_train, y_train)
X_test_A, y_test_A = get_data_without_5_6(X_test, y_test)
X_train_A, X_val_A = X_train_A[:30000], X_train_A[30000:]
y_train_A, y_val_A = y_train_A[:30000], y_train_A[30000:]

In [2]:


model_A = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(300, activation="relu"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(100, activation="relu"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(8, activation="softmax")
])
model_A.compile(loss="sparse_categorical_crossentropy", 
                optimizer="sgd", 
                metrics="accuracy")
model_A.fit(X_train_A, y_train_A, epochs=100, 
            validation_data=(X_val_A, y_val_A), 
            callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                     patience=10, 
                                                     restore_best_weights=True)])

Epoch 1/100
938/938 [==============================] - 7s 5ms/step - loss: 0.3977 - accuracy: 0.8659 - val_loss: 0.2692 - val_accuracy: 0.9088
Epoch 2/100
938/938 [==============================] - 5s 5ms/step - loss: 0.2771 - accuracy: 0.9043 - val_loss: 0.2425 - val_accuracy: 0.9173
Epoch 3/100
938/938 [==============================] - 3s 3ms/step - loss: 0.2402 - accuracy: 0.9177 - val_loss: 0.2338 - val_accuracy: 0.9207
Epoch 4/100
938/938 [==============================] - 3s 3ms/step - loss: 0.2150 - accuracy: 0.9246 - val_loss: 0.2207 - val_accuracy: 0.9251
Epoch 5/100
938/938 [==============================] - 3s 3ms/step - loss: 0.2044 - accuracy: 0.9281 - val_loss: 0.2209 - val_accuracy: 0.9233
Epoch 6/100
938/938 [==============================] - 3s 4ms/step - loss: 0.1892 - accuracy: 0.9326 - val_loss: 0.2141 - val_accuracy: 0.9277
Epoch 7/100
938/938 [==============================] - 3s 3ms/step - loss: 0.1761 - accuracy: 0.9371 - val_loss: 0.2168 - val_accuracy: 0.9251

In [3]:
model_A.evaluate(X_test_A, y_test_A)

250/250 [==============================] - 1s 2ms/step - loss: 0.2272 - accuracy: 0.9243


[0.22721777856349945, 0.9242500066757202]

In [4]:
def get_data_withonly_5_6(data, labels):
    new_data, new_labels = [], []
    for i, label in enumerate(labels):
        if label == 5:
            new_data.append(data[i])
            new_labels.append(1)
        elif label == 6:
            new_data.append(data[i])
            new_labels.append(0)
    return np.array(new_data), np.array(new_labels)

X_train_B, y_train_B = get_data_withonly_5_6(X_train, y_train)
X_test_B, y_test_B = get_data_withonly_5_6(X_test, y_test)
X_train_B, X_val_B = X_train_B[:10000], X_train_B[10000:]
y_train_B, y_val_B = y_train_B[:10000], y_train_B[10000:]

In [5]:
model_B = keras.Sequential([
    keras.layers.Flatten(input_shape=(28, 28)), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(300, activation="relu"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(100, activation="relu"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(10, activation="relu"), 
    keras.layers.BatchNormalization(), 
    keras.layers.Dense(1, activation="sigmoid")
])
model_B.compile(loss="binary_crossentropy", 
                optimizer="sgd", 
                metrics="accuracy")
model_B.fit(X_train_B, y_train_B, epochs=100, 
            validation_data=(X_val_B, y_val_B), 
            callbacks=[keras.callbacks.EarlyStopping(monitor="val_loss", 
                                                     patience=10, 
                                                     restore_best_weights=True)])

Epoch 1/100
313/313 [==============================] - 2s 5ms/step - loss: 0.1196 - accuracy: 0.9791 - val_loss: 0.0525 - val_accuracy: 0.9920
Epoch 2/100
313/313 [==============================] - 1s 3ms/step - loss: 0.0477 - accuracy: 0.9923 - val_loss: 0.0312 - val_accuracy: 0.9925
Epoch 3/100
313/313 [==============================] - 1s 4ms/step - loss: 0.0296 - accuracy: 0.9956 - val_loss: 0.0316 - val_accuracy: 0.9930
Epoch 4/100
313/313 [==============================] - 1s 4ms/step - loss: 0.0199 - accuracy: 0.9970 - val_loss: 0.0238 - val_accuracy: 0.9950
Epoch 5/100
313/313 [==============================] - 1s 3ms/step - loss: 0.0184 - accuracy: 0.9965 - val_loss: 0.0205 - val_accuracy: 0.9950
Epoch 6/100
313/313 [==============================] - 1s 3ms/step - loss: 0.0148 - accuracy: 0.9980 - val_loss: 0.0240 - val_accuracy: 0.9950
Epoch 7/100
313/313 [==============================] - 1s 3ms/step - loss: 0.0123 - accuracy: 0.9979 - val_loss: 0.0163 - val_accuracy: 0.9955

In [6]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 2ms/step - loss: 0.0076 - accuracy: 0.9985


[0.0075619034469127655, 0.9984999895095825]

In [7]:
model_A.evaluate(X_test_A, y_test_A)

250/250 [==============================] - 1s 2ms/step - loss: 0.2272 - accuracy: 0.9243


[0.22721777856349945, 0.9242500066757202]

In [8]:
model_B_on_A = keras.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [9]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [10]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False
model_B_on_A.compile(loss="binary_crossentropy", 
                     optimizer="sgd", 
                     metrics=["accuracy"])


In [11]:
history_b_on_A = model_B_on_A.fit(X_train_B, y_train_B, epochs=4, 
                                  validation_data=(X_val_B, y_val_B))

Epoch 1/4
313/313 [==============================] - 1s 3ms/step - loss: 0.1328 - accuracy: 0.9611 - val_loss: 0.0554 - val_accuracy: 0.9880
Epoch 2/4
313/313 [==============================] - 1s 3ms/step - loss: 0.0345 - accuracy: 0.9950 - val_loss: 0.0379 - val_accuracy: 0.9905
Epoch 3/4
313/313 [==============================] - 1s 2ms/step - loss: 0.0238 - accuracy: 0.9962 - val_loss: 0.0313 - val_accuracy: 0.9910
Epoch 4/4
313/313 [==============================] - 1s 2ms/step - loss: 0.0191 - accuracy: 0.9967 - val_loss: 0.0280 - val_accuracy: 0.9910


In [12]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = keras.optimizers.SGD(learning_rate=1e-4)
model_B_on_A.compile(loss="binary_crossentropy", 
                     optimizer=optimizer, 
                     metrics=["accuracy"])

history_b_on_A = model_B_on_A.fit(X_train_B, y_train_B, epochs=16, 
                                  validation_data=(X_val_B, y_val_B))

Epoch 1/16
313/313 [==============================] - 2s 4ms/step - loss: 0.0403 - accuracy: 0.9886 - val_loss: 0.0410 - val_accuracy: 0.9905
Epoch 2/16
313/313 [==============================] - 1s 4ms/step - loss: 0.0333 - accuracy: 0.9916 - val_loss: 0.0398 - val_accuracy: 0.9915
Epoch 3/16
313/313 [==============================] - 1s 4ms/step - loss: 0.0332 - accuracy: 0.9919 - val_loss: 0.0361 - val_accuracy: 0.9915
Epoch 4/16
313/313 [==============================] - 2s 5ms/step - loss: 0.0327 - accuracy: 0.9916 - val_loss: 0.0353 - val_accuracy: 0.9910
Epoch 5/16
313/313 [==============================] - 2s 7ms/step - loss: 0.0334 - accuracy: 0.9917 - val_loss: 0.0327 - val_accuracy: 0.9910
Epoch 6/16
313/313 [==============================] - 2s 6ms/step - loss: 0.0312 - accuracy: 0.9919 - val_loss: 0.0321 - val_accuracy: 0.9915
Epoch 7/16
313/313 [==============================] - 2s 7ms/step - loss: 0.0268 - accuracy: 0.9946 - val_loss: 0.0312 - val_accuracy: 0.9915
Epoch 

In [13]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.0151 - accuracy: 0.9975


[0.015052733942866325, 0.9975000023841858]

## Faster Optimizer

In [4]:
from tensorflow import keras
momentum_optimizer = keras.optimizers.SGD(learning_rate=0.001, 
                                          momentum=0.9)

In [2]:
from tensorflow import keras
nag_optimizer = keras.optimizers.SGD(learning_rate=1e-3, 
                                     momentum=0.9, 
                                     nesterov=True)

In [3]:
adagrad_optimizer = keras.optimizers.Adagrad(learning_rate=1e-3, 
                                             initial_accumulator_value=0.1, 
                                             epsilon=1e-10)

In [4]:
rmsprop_optimizer = keras.optimizers.RMSprop(learning_rate=1e-3, 
                                             rho=0.9)

In [5]:
adam_optimizer = keras.optimizers.Adam(learning_rate=0.001, 
                                       beta_1=0.9, 
                                       beta_2=0.999)

In [1]:
from tensorflow import keras
optimizer = keras.optimizers.SGD(learning_rate=0.01, decay=1e-4)

In [2]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch/s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(lr0=0.01, s=20)
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [3]:
def piecewise_constant_fn(epoch):
    if epoch < 5:
        return 0.01
    elif epoch < 15:
        return 0.005
    else:
        return 0.001

In [4]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, 
                                                 patience=5)

In [5]:
length_of_dataset = 2000
s = 20 * length_of_dataset // 32
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)

## Avoiding Overfitting Through Regularization

In [1]:
from tensorflow import keras

l1_dense_layer = keras.layers.Dense(
    units=100, 
    activation="elu", 
    kernel_initializer="he_normal", 
    kernel_regularizer=keras.regularizers.l1(0.01)
)

In [2]:
l2_dense_layer = keras.layers.Dense(
    units=100, 
    activation="elu", 
    kernel_initializer="he_normal", 
    kernel_regularizer=keras.regularizers.l2(0.01)
)

In [4]:
l1_l2_dense_layer = keras.layers.Dense(
    units=100, 
    activation="elu", 
    kernel_initializer="he_normal", 
    kernel_regularizer=keras.regularizers.l1_l2(0.01)
)

In [5]:
from functools import partial

RegularizedDense = partial(
    keras.layers.Dense, 
    activation="elu", 
    kernel_initializer="he_normal", 
    kernel_regularizer=keras.regularizers.l2(0.01)
)

model = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), 
    RegularizedDense(300), 
    RegularizedDense(100), 
    RegularizedDense(10, activation="softmax", 
                     kernel_initializer="glorot_uniform")
])

In [6]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), 
    keras.layers.Dropout(rate=0.2), 
    keras.layers.Dense(300, activation="elu", 
                       kernel_initializer="he_normal"), 
    keras.layers.Dropout(rate=0.2), 
    keras.layers.Dense(100, activation="elu", 
                       kernel_initializer="he_normal"), 
    keras.layers.Dropout(rate=0.2), 
    keras.layers.Dense(10, activation="softmax")
])

In [10]:
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_val = X_train[:50000], X_train[50000:]
y_train, y_val = y_train[:50000], y_train[50000:]


model = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.2), 
    keras.layers.Dense(300, activation="relu", 
                        kernel_initializer="he_normal"), 
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(rate=0.2), 
    keras.layers.Dense(100, activation="relu", 
                        kernel_initializer="he_normal"), 
    keras.layers.Dropout(rate=0.2), 
    keras.layers.Dense(10, activation="softmax")
]) 
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, 
                                                 patience=5)
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer="sgd", 
              metrics=["accuracy"])
model.fit(X_train, y_train, epochs=100, 
          validation_data=(X_val, y_val), 
          callbacks=[keras.callbacks.EarlyStopping(patience=10, 
                                                   restore_best_weights=True)])


Epoch 1/100
1563/1563 [==============================] - 7s 3ms/step - loss: 0.7499 - accuracy: 0.7347 - val_loss: 0.4495 - val_accuracy: 0.8339
Epoch 2/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.5557 - accuracy: 0.8009 - val_loss: 0.4183 - val_accuracy: 0.8449
Epoch 3/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.5038 - accuracy: 0.8169 - val_loss: 0.3996 - val_accuracy: 0.8519
Epoch 4/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.4814 - accuracy: 0.8258 - val_loss: 0.3785 - val_accuracy: 0.8615
Epoch 5/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.4537 - accuracy: 0.8347 - val_loss: 0.3762 - val_accuracy: 0.8636
Epoch 6/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.4426 - accuracy: 0.8378 - val_loss: 0.3611 - val_accuracy: 0.8694
Epoch 7/100
1563/1563 [==============================] - 5s 3ms/step - loss: 0.4325 - accuracy: 0.8420 - val_loss: 0.3584 - val_ac

In [19]:
model.save("fashion_mnist.h5")

In [11]:
import numpy as np

with keras.backend.learning_phase_scope(1):
    y_probas = np.stack([model.predict(X_test) for sample in range(100)])
y_proba = y_probas.mean(axis=0)

C:\Users\msi1\AppData\Local\Programs\Python\Python38\lib\contextlib.py:113: UserWarning: `tf.keras.backend.learning_phase_scope` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  return next(self.gen)


In [39]:
np.round(model.predict(X_test[:1]), 3)

array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002, 0.   ,
        0.998]], dtype=float32)

In [44]:
np.round(y_probas[:, :1], 3)

array([[[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002, 0.   ,
         0.998]],

       [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002, 0.   ,
         0.998]],

       [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002, 0.   ,
         0.998]],

       [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002, 0.   ,
         0.998]],

       [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002, 0.   ,
         0.998]],

       [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002, 0.   ,
         0.998]],

       [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002, 0.   ,
         0.998]],

       [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002, 0.   ,
         0.998]],

       [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002, 0.   ,
         0.998]],

       [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002, 0.   ,
         0.998]],

       [[0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.002, 0.   ,
         0.998]],


In [36]:
np.array([X_test[1]]).shape

(1, 28, 28)

In [45]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [1]:
from tensorflow import keras

keras.layers.Dense(100, activation="elu", 
                   kernel_initializer="he_normal", 
                   kernel_constraint=keras.constraints.max_norm())